# Ejemplo 006

**Problema:** Queremos detectar cambios en una tabla de la base de datos compartida de Ingenier@, y leer todos los registros que se modificaron. 


#### Importamos paquetes necesarios

In [45]:
# Importación de paquetes requeridos
import requests
import json
from datetime import datetime

#### Funciones auxiliares

In [46]:
# Función auxiliar para visualizar los datos
def pp(obj):
    print(json.dumps(obj, indent=2))

#### Parámetros del script

In [47]:
# Base URL de la api de Ingenier@
iga_api_base_url = "http://ing-acc-movil01.personal.corp/ingenier@/symfony/public/index.php/api"

# Última sincronización: última vez que se ejecutó este script. 
# Debemos obtener todos los registros que se modificaron o agregaron posteriormente a este datetime.
last_sync_datetime = "2023-10-05 22:34:15.905169"

# Tabla en la base de datos compartida
shdb_category = "CLASE SITIO"
shdb_group = "SITE_CLASS"
shdb_table_name = "SITE_CLASS_MAIN"


#### Consulta del historial de cambios

In [48]:
# Nos interesan eventos que cumplan dos condiciones:
# 1. Sean posteriores al datetime de la ultima sincronización
# 2. Sean eventos del tipo EVENT_ADD o EVENT_UPDATE (no nos interesan los eventos de tipo EVENT_REMOVE)
payload = {
  "category": shdb_category,
  "group": shdb_group,
  "tableName": shdb_table_name,
  "page": 1,
  "itemsPerPage": 1000000000,
  "filterCriteria": [
    {
      "fieldName": "dateTime",
      "conditionType": "greaterThanOrEqual",
      "values": [
        last_sync_datetime
      ]
    },
    {
      "fieldName": "eventType",
      "conditionType": "in",
      "values": [
        "EVENT_ADD",
        "EVENT_UPDATE"
      ]
    }
  ],
  "orderCriteria": []
}

res = requests.post(iga_api_base_url + "/secure/shared-database/change-history", json=payload)
pp(res.json())


{
  "status": "success",
  "page": 1,
  "totalPages": 1,
  "totalItems": 15,
  "itemsPerPage": 1000000000,
  "count": 15,
  "data": [
    {
      "id": "126320",
      "dateTime": "2023-10-05 22:34:15.990155",
      "eventType": "EVENT_ADD",
      "username": "u500469",
      "entityId": "7934",
      "entityInfo": "{\"siteName\":\"A41-A776\",\"class\":\"D\"}"
    },
    {
      "id": "126319",
      "dateTime": "2023-10-05 22:34:15.985638",
      "eventType": "EVENT_ADD",
      "username": "u500469",
      "entityId": "7933",
      "entityInfo": "{\"siteName\":\"A41-A420\",\"class\":\"D\"}"
    },
    {
      "id": "126318",
      "dateTime": "2023-10-05 22:34:15.980815",
      "eventType": "EVENT_ADD",
      "username": "u500469",
      "entityId": "7932",
      "entityInfo": "{\"siteName\":\"A40-B781\",\"class\":\"D\"}"
    },
    {
      "id": "126317",
      "dateTime": "2023-10-05 22:34:15.975948",
      "eventType": "EVENT_ADD",
      "username": "u500469",
      "entityId": "79

In [49]:
change_history_events = res.json()["data"]

updated_site_names = [
  json.loads(e["entityInfo"])["siteName"]
  for e in change_history_events
]
# Removemos valores duplicados en caso de haberlos
updated_site_names = list(set(updated_site_names))

# Con eso obtenemos la lista de sitios que fueron modificados o agregados
pp(updated_site_names)

[
  "A41-A073",
  "A24-V039",
  "A41-A776",
  "A28-Q137",
  "A43-B162",
  "A41-A069",
  "A26-U121",
  "A41-B685",
  "SID_017",
  "SID_049",
  "A41-A437",
  "A41-A420",
  "A41-A419",
  "A29-B230",
  "A40-B781"
]


#### Consulta del dato "Clase" para los sitios que fueron actualizados

In [50]:
# Obtenemos datos de la tabla para los sitios modificados o agregados
payload = {
  "category": shdb_category,
  "group": shdb_group,
  "tableName": shdb_table_name,
  "page": 1,
  "itemsPerPage": 1000000000,
  "filterCriteria": [
    {
      "fieldName": "siteName",
      "conditionType": "in",
      "values": updated_site_names
    }
  ],
  "orderCriteria": [],
  "resolveCustomChoiceValues": True
}

res = requests.post(iga_api_base_url + "/secure/shared-database/select-data", json=payload)
pp(res.json())

# Los datos que debemos impactar en la base están en el campo "data" del payload
site_class_data = res.json()["data"]

# Luego de impactar los datos deberíamos actualizar 

{
  "status": "success",
  "page": 1,
  "totalPages": 1,
  "totalItems": 15,
  "itemsPerPage": 1000000000,
  "count": 15,
  "data": [
    {
      "id": 7921,
      "siteName": "A24-V039",
      "class": "D"
    },
    {
      "id": 7920,
      "siteName": "A26-U121",
      "class": "D"
    },
    {
      "id": 7929,
      "siteName": "A28-Q137",
      "class": "D"
    },
    {
      "id": 7928,
      "siteName": "A29-B230",
      "class": "C"
    },
    {
      "id": 7932,
      "siteName": "A40-B781",
      "class": "D"
    },
    {
      "id": 7924,
      "siteName": "A41-A069",
      "class": "D"
    },
    {
      "id": 7926,
      "siteName": "A41-A073",
      "class": "D"
    },
    {
      "id": 7922,
      "siteName": "A41-A419",
      "class": "D"
    },
    {
      "id": 7933,
      "siteName": "A41-A420",
      "class": "D"
    },
    {
      "id": 7927,
      "siteName": "A41-A437",
      "class": "D"
    },
    {
      "id": 7934,
      "siteName": "A41-A776",
      "class

In [51]:
# Luego de impactar los datos deberíamos actualizar el valor de last_sync_datetime con fecha 
# y hora actual y persistirlo antes de la próxima ejecución del script.
# Tener en cuenta que hay que respetar el formato requerido por el endpoint de change-history:

last_sync_datetime = datetime.now()
print(last_sync_datetime)


2023-10-17 17:06:26.671188
